In [292]:
# coding: utf-8

import _pickle as cPickle
#Загрузка даных:
file = open("file.txt", 'rb')
df = cPickle.load(file)
file.close()
i=230
print(df.data[i])


давно известно маркетолог использовать различный приём воздействие клиент владельцысупермаркет знать этот приём невозможно обойтись поэтому стараться можно чаща применятий практика автор рассматриваться основной приём воздействие потенциальный клиент супер маркета супермаркет маркетинг товар покупка метод воздействие маркетинговый приём


In [293]:
#Функция чистки текста:
import pymorphy2
import re

def clean_text(text):
    morph = pymorphy2.MorphAnalyzer()
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text) #deleting symbols  
    text = ' '.join(word for word in text.split() if len(word)>3)
    text = " ".join(morph.parse(word)[0].normal_form for word in text.split())
    
    return text

In [294]:
# Максимальное количество слов в самом длинном описании заявки
max_words = 0
for desc in df.data[0:i+1]:
    words = len(desc.split())
    if words > max_words:
        max_words = words
print('Максимальное количество слов в самом длинном описании заявки: {} слов'.format(max_words))


Максимальное количество слов в самом длинном описании заявки: 119 слов


In [295]:
import pandas as pd
import numpy as np
import tensorflow as tf
from collections import Counter
#Составление словаря
vocab = Counter()
newsgroups_train=df
for text in newsgroups_train.data[0:i+1]:
    for word in text.split(' '):
        vocab[word.lower()]=1
        
total_words = len(vocab)
print('Всего слов в словаре: ', total_words)
#Функция, которая возвращает индекс слова в словаре
def get_word_2_index(vocab):
    word2index = {}
    for i,word in enumerate(vocab):
        word2index[word.lower()] = i
        
    return word2index

word2index = get_word_2_index(vocab)

#Составление числовых векторов
batches = []
results = []
texts = newsgroups_train.data[0:i+1]
categories = newsgroups_train.target[0:i+1]

for text in texts:
   k=0
   layer = np.zeros(max_words,dtype=float)
   for word in text.split(' '):
        if (vocab[word.lower()])!=0:
          layer[k] = word2index[word.lower()]
          k+=1
            
   batches.append(layer)
X_train=np.array(batches)


y=0
num_classes=21
target=np.zeros((num_classes))
y_train=np.zeros((i+1,num_classes))
for у in range(i+1):
    y_train[y,df.target[y]-1]=1
    target[df.target[y]-1]+=1
    y+=1    

print('Распределение по рубликам: ',target)
import keras


Всего слов в словаре:  3731
Распределение по рубликам:  [13. 39. 25. 23.  6.  6. 19. 10. 22.  7.  4.  6.  2. 18.  6.  5.  6.  1.
  3.  8.  2.]


In [296]:
#Разбиение на обучающую и тестовую выборку
n=15
X_test=X_train[i+1-n:i+1] 
y_test=y_train[i+1-n:i+1]
y_train=y_train[0:i+1-n]
X_train=X_train[0:i+1-n]
print('Размерность обучающей выборки: ',X_train.shape)
print('Размерность тестовой выборки: ',X_test.shape)

Размерность обучающей выборки:  (216, 119)
Размерность тестовой выборки:  (15, 119)


In [297]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
#Реккурентная модель
# максимальное количество слов для анализа
max_features = total_words

print(u'Собираем модель...')
model = Sequential()
model.add(Embedding(total_words, embedding_vecor_length, input_length=max_words))
model.add(LSTM(100)
model.add(Dense(num_classes, activation='softmax'))#'sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print (model.summary())

Собираем модель...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 119, 32)           119392    
_________________________________________________________________
lstm_23 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dense_23 (Dense)             (None, 21)                1365      
Total params: 145,589
Trainable params: 145,589
Non-trainable params: 0
_________________________________________________________________
None


In [301]:
from keras.layers import Dropout,Conv1D,MaxPooling1D
#Сверточная модель
# максимальное количество слов для анализа
max_features = total_words

print(u'Собираем модель...')
model = Sequential()
embedding_vecor_length = 32
model.add(Embedding(total_words, embedding_vecor_length, input_length=max_words))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(keras.layers.Dropout(0.3))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(keras.layers.Dropout(0.3))
model.add(LSTM(200,dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
print(model.summary())

Собираем модель...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 119, 32)           119392    
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 119, 32)           3104      
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 59, 32)            0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 59, 32)            0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 59, 32)            4128      
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 29, 32)            0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 29, 32)            0 

In [302]:
#Обучение модели
batch_size = 64
epochs = 30

print(u'Тренируем модель...')
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))

Тренируем модель...
Train on 216 samples, validate on 15 samples
Epoch 1/30
216/216 [==============================] - 152s 705ms/step - loss: 3.0354 - acc: 0.1065 - val_loss: 3.0182 - val_acc: 0.0667
Epoch 2/30
216/216 [==============================] - 2s 9ms/step - loss: 2.9815 - acc: 0.1759 - val_loss: 2.9131 - val_acc: 0.0667
Epoch 3/30
216/216 [==============================] - 1s 3ms/step - loss: 2.8242 - acc: 0.1759 - val_loss: 2.8061 - val_acc: 0.0667
Epoch 4/30
216/216 [==============================] - 1s 2ms/step - loss: 2.7461 - acc: 0.1759 - val_loss: 2.7440 - val_acc: 0.0667
Epoch 5/30
216/216 [==============================] - 1s 3ms/step - loss: 2.7514 - acc: 0.1250 - val_loss: 2.7445 - val_acc: 0.1333
Epoch 6/30
216/216 [==============================] - 1s 3ms/step - loss: 2.7257 - acc: 0.1481 - val_loss: 2.7575 - val_acc: 0.0667
Epoch 7/30
216/216 [==============================] - 1s 3ms/step - loss: 2.7093 - acc: 0.1759 - val_loss: 2.8226 - val_acc: 0.0667
Epoch 8

In [303]:
#Оценка модели
score = model.evaluate(X_test, y_test,
                       batch_size=batch_size, verbose=1)
print()
print(u'Оценка теста: {}'.format(score[0]))
print(u'Оценка точности модели: {}'.format(score[1]))

15/15 [==============================] - 0s 1ms/step

Оценка теста: 3.161428213119507
Оценка точности модели: 0.13333334028720856


In [ ]:
#Получение прогноза от модели

prediction = model.predict(X_test)
 
j=0
for j in range(15):
    print('Правильная категория: ',np.argmax(y_test[j])+1)
    print("Определенная моделью категория: ",np.argmax(prediction[j])+1)